In [178]:
import pandas as pd 
import numpy as np




In [179]:
food_nutrition = pd.read_csv('./data/food_nutrition.csv')
food_nutrition.head(3)

,Unnamed: 0,water(g),energy(kcal),protein(g),fat(g),carb(g),fiber(g),sugar(g),calcium(mg),iron(mg),...,copper(mg),manganese(mg),selenium(ug),VC(mg),VB(mg),VA(ug),VD(ug),VK(ug),caffeine(mg),alcohol(mg)
0,lower,500.0,1200.0,0.8,0.20,130.0,25.0,2.0,1000,8.0,...,0.9,1.80,55.0,75.0,1.00,700,15,75.00,1,1
1,upper,5000.0,2400.0,2.0,0.35,300.0,70.0,36.0,2500,45.0,...,10.0,11.00,400.0,2000.0,100.00,3000,100,120.00,400,30000
2,watermelon,4130.0,1360.0,27.6,6.78,341.0,18.1,280.0,316,10.8,...,1.9,1.72,18.1,366.0,2.03,1270,0,4.52,0,0


In [180]:
M = food_nutrition[food_nutrition.columns[1:]][2:].to_numpy()   # remove the `name` column
print(M.shape)

(4, 24)


In [182]:
R = np.array([1, 1, 0.8, 0.6, 0.4])   # increment ratios at 20, 40, 60, 80, 100 seconds
R = 20 * R.cumsum()
print(R)
R.shape

[20. 40. 56. 68. 76.]


(5,)

In [183]:
# U[?] is the upper limit of each nutrition fact at time ?

U = []
for i in range(len(R)):
  U.append((1 + R[i]) * UPPER)

U = np.array(U)
# print(U)      # DEBUG
print(' 时间点个数 x 食物种类 x 营养成分种类: ', U.shape )


 时间点个数 x 食物种类 x 营养成分种类:  (5, 1, 24)


In [184]:
# L[?] is the lower limit of each nutrition fact at time ?

L = []
for i in range(len(R)):
  L.append((1 + R[i]) * LOWER)

L = np.array(L)

# print(L)      # DEBUG
print(' 时间点个数 x 食物种类 x 营养成分种类: ', L.shape )


 时间点个数 x 食物种类 x 营养成分种类:  (5, 1, 24)


In [185]:
# 假设每隔20秒，每种食物生成 X 个：

X = 5
G = np.reshape(np.array(
  [X for _ in range(len(R) * M.shape[0])]
), (len(R), M.shape[0]))

np.cumsum(G, axis=0, out=G)

print(G.shape)
G

(5, 4)


array([[ 5,  5,  5,  5],
       [10, 10, 10, 10],
       [15, 15, 15, 15],
       [20, 20, 20, 20],
       [25, 25, 25, 25]])

In [186]:
USER_ = np.matmul(G, M)

print(USER_.shape)

(5, 24)


In [26]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


# Load the data from the CSV file
file_path = './data/food_nutrition.csv'
nutrition_data = pd.read_csv(file_path)

# Display the data to understand its structure and contents
nutrition_data.head()
nutrition_data.drop(['unit'], axis=1, inplace=True)       # drop the `unit` column
nutrition_data = nutrition_data.reset_index(drop=True)


# Separate the nutrition data for the foods and the lower/upper bounds
food_nutrition = nutrition_data.iloc[2:]
lower_bounds = nutrition_data.iloc[0, 1:]
upper_bounds = nutrition_data.iloc[1, 1:]

# Convert the bounds to a dataframe for easier manipulation
bounds = pd.concat([lower_bounds, upper_bounds], axis=1).transpose()
bounds.index = ['lower', 'upper']

# Adjust the protein and fat bounds according to the given rules (body weight = 60kg)
bounds.loc['lower', 'protein(g)'] *= 60  # Lower bound of protein
bounds.loc['upper', 'protein(g)'] *= 60  # Upper bound of protein
bounds.loc['lower', 'fat(g)'] = bounds.loc['lower', 'energy(kcal)'] * 0.20  # Lower bound of fat (20% of kcal)
bounds.loc['upper', 'fat(g)'] = bounds.loc['upper', 'energy(kcal)'] * 0.35  # Upper bound of fat (35% of kcal)

# Define the multipliers for each checkpoint based on the seconds passed
checkpoints = [20, 40, 56, 68, 76]

# Since the multipliers are not absolute and can be part of the value planning,
# for now, we will create a basic function to update the bounds based on the current second
def update_bounds(seconds, initial_bounds):
    multiplier = 1
    if seconds in checkpoints:
        multiplier = seconds / 20
    return initial_bounds * multiplier

# Define a function to simulate the game
def simulate_game(food_nutrition, bounds, checkpoints, total_seconds=120, weight=60):
    # Initialize player's nutrition state
    player_state = pd.Series(0, index=bounds.columns)
    
    # Initialize the checkpoints
    checkpoint_bounds = {cp: update_bounds(cp, bounds) for cp in checkpoints}
    
    # Simulate the food generation and player's choices
    foods_selected = []
    for second in range(0, total_seconds):
        # Randomly select a food every second
        selected_food = food_nutrition.sample(n=1)
        foods_selected.append(selected_food.index[0])
        
        # Update the player's nutrition state
        for nutrient in player_state.index:
            player_state[nutrient] += selected_food[nutrient].values[0]
        
        # Check the player's state at each checkpoint
        if second in checkpoints:
            for nutrient in player_state.index:
                # If the player is outside the bounds, return failure and the second of failure
                if player_state[nutrient] > checkpoint_bounds[second].loc['upper', nutrient]:
                  print('over nutrition')
                  return False, second, foods_selected, player_state
                if player_state[nutrient] < checkpoint_bounds[second].loc['lower', nutrient]:
                  print('mal nutrition')
                  return False, second, foods_selected, player_state
                
                    
    # If the player survives all checkpoints, return success
    return True, total_seconds, foods_selected, player_state

# Run the simulation once to test the setup
simulate_game(food_nutrition, bounds, checkpoints)


over nutrition


(False,
 20,
 [17, 3, 3, 9, 12, 3, 18, 4, 21, 3, 9, 3, 15, 22, 14, 20, 11, 15, 4, 21, 5],
 water(g)           1220.27
 energy(kcal)       4310.00
 protein(g)          903.53
 fat(g)              160.37
 carb(g)             385.20
 fiber(g)             51.20
 sugar(g)            131.14
 calcium(mg)        1411.00
 iron(mg)             41.30
 magnesium(mg)       494.10
 phosphorus(mg)     3385.00
 potassium(mg)      3688.00
 sodium(mg)         3446.00
 zinc(mg)             36.06
 copper(mg)           70.52
 manganese(mg)        12.05
 selenium(ug)        257.20
 VC(mg)               69.23
 VB(mg)                3.08
 VA(ug)              125.00
 VD(ug)               11.90
 VK(ug)               43.90
 caffeine(mg)          0.00
 alcohol(mg)       69900.00
 dtype: float64)

# 游戏营养成分线性规划分析

在我们的游戏设计中，玩家需要在120秒内选择各种食物以满足特定的营养需求。每20秒，玩家的营养状态会进行一次检查，以确保所有营养成分都处于健康的范围内。我们使用线性规划来确定玩家在第20秒时可以选择的食物组合。

## 线性规划设置

### 变量定义

- 设 \( x_1, x_2, x_3, x_4 \) 分别代表苹果、香蕉、牛油果、西瓜被选择的次数。

### 约束条件

* 对于每种营养成分，如能量（energy），苹果、香蕉、牛油果、西瓜的能量值分别为 \( e_1, e_2, e_3, e_4 \)。则他们的线性组合 \( e_1x_1 + e_2x_2 + e_3x_3 + e_4x_4 \) 必须满足在第20秒的能量下限 \( L_e \) 和上限 \( U_e \) 之间。
- 类似的约束需要为所有24种营养成分设置。

### 目标函数

- 因为我们的目标是找到任何满足所有约束条件的食物组合，我们可以简单地设置目标函数为 \( x_1 + x_2 + x_3 + x_4 \)，表示食物选择的总次数。实际上，这个目标函数在这里不是最优化的目标，而是用于找到一个可行解。

## 求解过程

使用`scipy.optimize.linprog`方法，我们提供了上述变量、约束条件和目标函数来求解线性规划问题。求解过程如下：

1. 从食物营养数据库中提取约束系数。
2. 根据20秒检查点的营养成分下限和上限设置约束条件。
3. 定义每种食物选择次数的变量范围。
4. 执行线性规划求解。

## 结果分析

线性规划求解结果表明，在当前的游戏数值设定下，没有可行的解决方案。这意味着玩家无法在第20秒时达到所有营养成分的健康范围。因此，我们需要对游戏的数值设计进行调整，以确保至少存在一个可行的解决方案，让玩家有机会成功通过第20秒的营养检查。


In [27]:
import numpy as np
from scipy.optimize import linprog

# Get the bounds for the 20-second checkpoint
bounds_20 = update_bounds(20, bounds)

# Extract the nutritional information of each food for the linear programming constraints
food_constraints = food_nutrition.set_index('name').T

# The lower and upper bounds for the nutritional information at the 20-second checkpoint
lower_bounds_20 = bounds_20.loc['lower']
upper_bounds_20 = bounds_20.loc['upper']

# The number of selections of each food is our variable, which cannot be negative
x_bounds = [(0, None) for _ in range(len(food_nutrition))]

# The objective function is arbitrary since we just want to find a feasible solution
# Here we use a vector of ones to simply sum the selections, but the actual values do not matter
c = [1 for _ in range(len(food_nutrition))]

# The constraints are that for each nutrient, the total amount must be between the lower and upper bounds
A = food_constraints.values  # Coefficients for the constraints
b_lower = lower_bounds_20.values  # Lower bounds for the constraints
b_upper = upper_bounds_20.values  # Upper bounds for the constraints

# The bounds are set in a way that the total for each nutrient must be less than the upper bound and more than the lower bound
# We create two sets of constraints: one for the lower bound and one for the upper bound
A_ub = A  # Coefficients for the upper bound constraints
b_ub = b_upper  # Right-hand side for the upper bound constraints

A_lb = -A  # Coefficients for the lower bound constraints (negated to convert to <= form)
b_lb = -b_lower  # Right-hand side for the lower bound constraints (negated to convert to <= form)

# Combine the upper and lower bound constraints
A_combined = np.vstack([A_ub, A_lb])
b_combined = np.concatenate([b_ub, b_lb])

# Solve the linear programming problem
result = linprog(c, A_ub=A_combined, b_ub=b_combined, bounds=x_bounds, method='highs')

result_x = result.x if result.success else None
result_message = result.message

(result.success, result_x, result_message)


(False,
 None,
 'The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)')